In [1]:
import sys 
import os
lib_path = './lib'
sys.path.insert(0, lib_path)     ## add lib path to sys path 
import pandas as pd
import MultiContagion as mc
import igraph
import random
import numpy as np
from matplotlib import pylab as plt
import scipy.stats
import copy
from scipy.sparse import linalg as LA
#import powerlaw
%matplotlib inline

In [2]:
year = 2016

In [3]:
import_path = "./data/adjacency_matrix/"
export_path = './results/centralities/'

In [4]:
#import the aggregate adjacency matrix
aggregate_am = np.genfromtxt (import_path + 'AM4_all_nodes_aggregate'+str(year)+'.csv', delimiter=",")
df_names = pd.read_csv(import_path+ 'all_country_name4.csv', header=None)
names = list(df_names[0])
Aggregate_g = igraph.Graph.Weighted_Adjacency(list(np.transpose(aggregate_am)))
Aggregate_g.vs["name"] = copy.deepcopy(names)

In [5]:
#Import the adjacency matrix and make graphs of all the layers.
cdis_equity_am = np.genfromtxt (import_path +'AM4_all_nodesCDIS-equity'+str(year)+'.csv', delimiter=",")
cdis_equity_g = igraph.Graph.Weighted_Adjacency(list(np.transpose(cdis_equity_am)))
cdis_equity_g.vs["name"] = copy.deepcopy(names)

cdis_debt_am = np.genfromtxt (import_path +'AM4_all_nodesCDIS-debt'+str(year)+'.csv', delimiter=",")
cdis_debt_g = igraph.Graph.Weighted_Adjacency(list(np.transpose(cdis_debt_am)))
cdis_debt_g.vs["name"] = copy.deepcopy(names)

cpis_equity_am = np.genfromtxt (import_path +'AM4_all_nodesCPIS-equity'+str(year)+'.csv', delimiter=",")
cpis_equity_g = igraph.Graph.Weighted_Adjacency(list(np.transpose(cpis_equity_am)))
cpis_equity_g.vs["name"] = copy.deepcopy(names)

cpis_debt_am = np.genfromtxt (import_path +'AM4_all_nodesCPIS-debt'+str(year)+'.csv', delimiter=",")
cpis_debt_g = igraph.Graph.Weighted_Adjacency(list(np.transpose(cpis_debt_am)))
cpis_debt_g.vs["name"] = copy.deepcopy(names)

bis_am = np.genfromtxt (import_path +'AM4_all_nodesBIS'+str(year)+'.csv', delimiter=",")
bis_g = igraph.Graph.Weighted_Adjacency(list(np.transpose(bis_am)))
bis_g.vs["name"] = copy.deepcopy(names)

In [6]:
G_list = [cdis_equity_g, cdis_debt_g, cpis_equity_g, cpis_debt_g, bis_g ]

In [7]:
AM_list = [np.transpose(cdis_equity_am), np.transpose(cdis_debt_am), np.transpose(cpis_equity_am), np.transpose(cpis_debt_am), np.transpose(bis_am) ]

In [8]:
In_list = [ np.array(g.strength(mode ="IN", weights= g.es["weight"])) for g in G_list]
Out_list = [ np.array(g.strength(mode ="OUT", weights= g.es["weight"])) for g in G_list]

In [9]:
def make_supra_coup(list_adj, list_vector):
    "Takes a list of adjacency matrix and list of vectors to use as coupling for layers"
    n_layers = len(list_adj)
    dim = list_adj[0].shape[0]
    sup_list = []
    for i in range(n_layers):
        row = []
        for j in range(n_layers):
            if i == j:
                row.append(list_adj[i])
            else:
                row.append(np.diag(list_vector[j]))
        sup_list.append(row)
    return np.bmat(sup_list)

# For real

In [10]:
supra_adj = make_supra_coup(AM_list, In_list)
G_supra = igraph.Graph.Weighted_Adjacency(supra_adj.tolist())
complete_multi_pr = G_supra.personalized_pagerank(weights = G_supra.es["weight"])

In [11]:
G_supra = igraph.Graph.Weighted_Adjacency(supra_adj.tolist())

In [12]:
G_supra = igraph.Graph.Weighted_Adjacency(supra_adj.tolist())

In [13]:
G_supra.is_weighted(), G_supra.is_directed()

(True, True)

In [14]:
def save_centrality(y, namefile, namecent , country, cent):
    f = open(export_path+namefile+str(y) +".csv", "w")
    f.write("Country,"+namecent+"\n")
    n = len(country)
    for i in range(n):
        f.write(str(country[i]) + "," + str(cent[i])+ "\n")
    f.close()

## PageRank

In [15]:
pr_agg = Aggregate_g.personalized_pagerank(weights = Aggregate_g.es["weight"], damping = 0.85)
node_names = np.array(copy.deepcopy(names))
inds = np.array(pr_agg).argsort()[::-1][:]
sort_names = node_names[inds]
pr_agg_ord = np.array(pr_agg)[inds]

In [16]:
sort_names[:30]

array(['United States', 'United Kingdom', 'Luxembourg', 'Netherlands',
       'France', 'Cayman Islands', 'Germany', 'Japan', 'Switzerland',
       'Ireland', 'Canada', 'China  P.R.: Hong Kong',
       'China  P.R.: Mainland', 'Italy', 'Spain', 'Australia', 'Belgium',
       'Singapore', 'Bermuda', 'India', 'Brazil', 'Sweden', 'Austria',
       'Korea  Republic of', 'Mexico', 'Virgin Islands  British',
       'Russian Federation', 'Jersey', 'Denmark',
       'Taiwan Province of China'], dtype='<U36')

In [17]:
save_centrality(year, "Agg_PageRankT", "PageRank", sort_names, pr_agg_ord)

In [18]:
def get_pagerank_list(G, n_layers = 5, names= names, r = 0.85):
    pr = G.personalized_pagerank(weights = G.es["weight"], damping = r)
    multi_pr = []
    n_nodes = int((len(G.degree()))/5)
    multi_pr = []
    for i in range(n_nodes):
        cent = 0
        for k in range(n_layers):
            cent += pr[i + n_nodes*k] 
        multi_pr.append(cent)
    node_names = np.array(copy.deepcopy(names))
    inds = np.array(multi_pr).argsort()[::-1][:]
    sort_names_multi_pr = node_names[inds]
    multi_sort_pr = np.array(multi_pr)[inds]
    return sort_names_multi_pr, multi_sort_pr

In [19]:
n_pr, num_pr = get_pagerank_list(G_supra, r = 0.85)

In [20]:
n_pr, num_pr = get_pagerank_list(G_supra, r = 0.85)
n_pr[:30]

array(['United States', 'United Kingdom', 'France', 'Switzerland',
       'Luxembourg', 'Netherlands', 'India', 'Ireland', 'Cayman Islands',
       'Spain', 'Germany', 'China  P.R.: Mainland', 'Australia',
       'China  P.R.: Hong Kong', 'Italy', 'Canada', 'Japan',
       'Virgin Islands  British', 'Brazil', 'Indonesia', 'Belgium',
       'Peru', 'Curacao', 'Turkey', 'Singapore', 'Mexico', 'South Africa',
       'Korea  Republic of', 'Taiwan Province of China', 'Sweden'],
      dtype='<U36')

In [21]:
def make_supra_coup2(list_adj, list_vector):
    "Takes a list of adjacency matrix and list of vectors to use as coupling for layers"
    n_layers = len(list_adj)
    dim = list_adj[0].shape[0]
    sup_list = []
    for i in range(n_layers):
        row = []
        for j in range(n_layers):
            if i == j:
                row.append(list_adj[i])
            else:
                row.append(np.diag(list_vector[i]))
        sup_list.append(row)
    return np.bmat(sup_list)

In [22]:
supra_adj2 = make_supra_coup2(AM_list, In_list)
G_supra2 = igraph.Graph.Weighted_Adjacency(supra_adj2.tolist())

In [23]:
n_pr2, num_pr2 = get_pagerank_list(G_supra2)

In [24]:
n_pr2;

In [25]:
n_pr2, num_pr2 = get_pagerank_list(G_supra2, r = 0.85)
n_pr2[:30]

array(['United States', 'United Kingdom', 'France', 'Luxembourg',
       'Netherlands', 'Ireland', 'Switzerland', 'Germany', 'Japan',
       'Cayman Islands', 'India', 'Australia', 'Spain', 'Brazil',
       'China  P.R.: Hong Kong', 'China  P.R.: Mainland', 'Canada',
       'Italy', 'Belgium', 'Virgin Islands  British', 'Mexico',
       'Indonesia', 'Turkey', 'Bahrain  Kingdom of', 'Panama', 'Sweden',
       'Curacao', 'Singapore', 'Malaysia', 'Russian Federation'],
      dtype='<U36')

In [26]:
save_centrality(year, "MultCoupCashFlow_PageRankT", "PageRank centrality", n_pr, num_pr)

In [27]:
save_centrality(year, "MultCoupRiskFlow_PageRankT", "PageRank centrality", n_pr2, num_pr2)

## Eigenvector

In [28]:
def get_eigenvector_list(G, n_layers = 5, names= names):
    pr = G.eigenvector_centrality(weights = G.es["weight"])
    multi_pr = []
    n_nodes = int((len(G.degree()))/5)
    multi_pr = []
    for i in range(n_nodes):
        cent = 0
        for k in range(n_layers):
            cent += pr[i + n_nodes*k] 
        multi_pr.append(cent)
    node_names = np.array(copy.deepcopy(names))
    inds = np.array(multi_pr).argsort()[::-1][:]
    sort_names_multi_pr = node_names[inds]
    multi_sort_pr = np.array(multi_pr)[inds]
    return sort_names_multi_pr, multi_sort_pr

In [29]:
n_ei, num_ei = get_eigenvector_list(G_supra)
n_ei2, num_ei2 = get_eigenvector_list(G_supra2)

D:\data\chengyu\_GFF\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: Weighted directed graph in eigenvector centrality at src\centrality.c:362
  


In [30]:
n_ei2;

In [31]:
save_centrality(year, "MultCoupCashFlow_EigenvectorT", "Eigenvector centrality", n_ei, num_ei)
save_centrality(year, "MultCoupRiskFlow_EigenvectorT", "Eigenvector centrality", n_ei2, num_ei2)

## Hubs and Auth

In [32]:
def G_list_hub(G_adj_list):
    G_hub = []
    for g in G_adj_list:
        new = np.dot(g, g.transpose())
        G_hub.append(new)   
    return G_hub

In [33]:
def G_list_auth(G_adj_list):
    G_hub = []
    for g in G_adj_list:
        new = np.dot( g.transpose(), g)
        G_hub.append(new)   
    return G_hub

In [34]:
def hub_auth_diag(Coup_list):
    coup = []
    for c in Coup_list:
        coup.append(c*c)
    return coup

In [35]:
G_supra_hub = make_supra_coup(G_list_hub(AM_list), hub_auth_diag(In_list))
G_supra_auth = make_supra_coup(G_list_auth(AM_list), hub_auth_diag(In_list))

In [36]:
G_supra_hub[0].shape

(1, 1065)

In [37]:
def get_hub_list(AM, n_layers = 5, names= names):
    hub_eig = LA.eigs(AM, k=1, which='LM')[1]
    multi_pr = []
    #print hub_eig[0]
    n_nodes = int((AM[0].shape[1])/5.)
    multi_pr = []
    for i in range(n_nodes):
        cent = 0
        for k in range(n_layers):
            
            cent += abs(hub_eig[i + n_nodes*k])
        
        multi_pr.append(cent[0])
    
    node_names = np.array(copy.deepcopy(names))
    inds = np.array(multi_pr).argsort()[::-1][:]
    sort_names_multi_pr = node_names[inds]
    multi_sort_pr = np.array(multi_pr)[inds]
    return sort_names_multi_pr, multi_sort_pr

In [38]:
n_hub, num_hub = get_hub_list(G_supra_hub)
n_auth, num_auth = get_hub_list(G_supra_auth)

In [39]:
n_auth, num_auth = get_hub_list(G_supra_auth)

In [40]:
n_auth[:20]

array(['United States', 'United Kingdom', 'Netherlands', 'France',
       'Luxembourg', 'Germany', 'Ireland', 'Cayman Islands',
       'Switzerland', 'Japan', 'Italy', 'Belgium', 'Canada', 'Australia',
       'Spain', 'China  P.R.: Hong Kong', 'China  P.R.: Mainland',
       'Brazil', 'Bermuda', 'Singapore'], dtype='<U36')

In [41]:
n_hub[:20]

array(['United States', 'Luxembourg', 'Netherlands', 'United Kingdom',
       'Germany', 'Japan', 'France', 'China  P.R.: Hong Kong',
       'Switzerland', 'Bermuda', 'Belgium', 'Cayman Islands', 'Canada',
       'Italy', 'Ireland', 'Norway', 'Jersey', 'Singapore', 'Australia',
       'China  P.R.: Mainland'], dtype='<U36')

In [42]:
#save_centrality(year, "MultCoupCashFlow_Hub", "Hub Score", n_hub, num_hub)
#save_centrality(year, "MultCoupCashFlow_Auth", "Authority Score", n_auth, num_auth)

### other coup

In [43]:
G_supra_hub2 = make_supra_coup2(G_list_hub(AM_list), hub_auth_diag(In_list))
G_supra_auth2 = make_supra_coup2(G_list_auth(AM_list), hub_auth_diag(In_list))

In [44]:
n_hub2, num_hub2 = get_hub_list(G_supra_hub2)
n_auth2, num_auth2 = get_hub_list(G_supra_auth2)

In [45]:
#n_hub2, num_hub2 = get_eigenvector_list(igraph.Graph.Weighted_Adjacency(G_supra_hub2.tolist()))
#n_auth2, num_auth2 = get_eigenvector_list(igraph.Graph.Weighted_Adjacency(G_supra_auth2.tolist()))

In [46]:
n_auth2[:30]

array(['United States', 'United Kingdom', 'Netherlands', 'France',
       'Germany', 'Cayman Islands', 'Luxembourg', 'Ireland', 'Japan',
       'Switzerland', 'Italy', 'Australia', 'Canada', 'Belgium', 'Spain',
       'China  P.R.: Hong Kong', 'Sweden', 'China  P.R.: Mainland',
       'Brazil', 'Bermuda', 'Singapore', 'Austria', 'Korea  Republic of',
       'Mexico', 'Jersey', 'Denmark', 'Norway', 'India',
       'Virgin Islands  British', 'Finland'], dtype='<U36')

In [47]:
n_hub2[:30]

array(['United States', 'Luxembourg', 'Japan', 'United Kingdom',
       'Netherlands', 'Germany', 'France', 'China  P.R.: Hong Kong',
       'Switzerland', 'Bermuda', 'Canada', 'Cayman Islands', 'Belgium',
       'Italy', 'Norway', 'Singapore', 'Australia', 'Ireland', 'Spain',
       'China  P.R.: Mainland', 'Jersey', 'Sweden', 'Guernsey', 'Denmark',
       'South Africa', 'Austria', 'Finland', 'Gibraltar',
       'Korea  Republic of', 'Saudi Arabia'], dtype='<U36')

In [48]:
save_centrality(year, "MultCoupRiskFlow_HubT", "Hub Score", n_hub2, num_hub2)
save_centrality(year, "MultCoupRiskFlow_AuthT", "Authority Score", n_auth2, num_auth2)

In [49]:
num_hub2 [:20]

array([1.91177565e+00, 1.30278977e-02, 8.30098336e-03, 8.01173253e-03,
       7.49938244e-03, 6.69661777e-03, 5.35692687e-03, 4.81521296e-03,
       3.52227223e-03, 2.44480141e-03, 2.26144665e-03, 2.20983673e-03,
       2.19272982e-03, 2.12481683e-03, 1.75667614e-03, 1.32512502e-03,
       1.30904372e-03, 1.21826042e-03, 1.06248994e-03, 1.04083922e-03])